# copy of GLS from case 1 to easier combine files later and have consistent notation

In [1]:
type ols_results
  coefs 
  yhat
  res
  vcv
  tstat
  pval
end

In [2]:
Int(floor(3.1))

3

In [3]:
# keyword arguments are placed after semicolon
function gls(y, X; corr=:none, lags=nothing)
    
    # more stable: β̂ = X \ y, see notes at bottom
    β̂ = inv(X' * X) * (X' * y)
    ŷ = X * β̂
    μ̂ = y - ŷ
    
    T, K = size(X)
    σ̂² = dot(μ̂, μ̂) / (T - K)

    #use correction for variance covariance
    if corr == :none
        vcv = σ̂² * inv(X' * X)
        
    elseif corr == :white
        # or do newey_west with lags=0
        vcv = inv(X' * X) * X' * diagm(μ̂.^2) * X * inv(X' * X)
        
    elseif corr == :newey_west
        if lags == nothing 
            lags = Int(floor(T^(1/4)))
        end
        vcv = newey_west(X, μ̂, lags)
    else
        error("")
    end

    # T statistics for H₀: β₀ = 0
    tstat = β̂ ./ sqrt(diag(vcv))
    
    # absolute value and times two for double sided test
    pval  = 2 * ccdf(TDist(T-K), abs(tstat)) 

    return ols_results(β̂, ŷ, μ̂, vcv, tstat, pval)
end

gls (generic function with 1 method)

Strict set of GM assumptions:
* X is deterministic, x is thus fixed over repeated samples
* errors $\mu$ are normally distributed with assumed homoscedastic errors

Give the small sample and asymptotic properties of the OLS estimator for $\beta$ and for the estimator of the standard errors.

Small sample properties:
* best unbiased estimator
* estimator is normally distributed (stems from the fact that $\hat{\beta}$ is linear function of disturbance vector $\mu$)
* covariance matrix $\sigma^2(X'X)^{-1}$ with an unbiased estimator of $\sigma^2$ given by

$$\hat{\sigma}^2 = \frac{\hat{\mu}'\hat{\mu}}{N-K} = \frac{y'My}{N-K}$$


Asymptotic properties:
* same under the GM conditions
* $\bar{x}_N$ assymptotically approaches $N(\mu,\frac{\sigma^2}{N})$

In [4]:
using Distributions

# Setting up a monte carlo simulation

In [5]:
β₀ = 10
β₁ = 1
β = [β₀, β₁]
σ² = 1
sample_size = 25
R = 100000

100000

In [6]:
#Model to generate Y
y(X,β,μ) = X * β + μ

y (generic function with 1 method)

Step 1: specify a population = N(5,2) draw sample once to have a deterministic sample, sample errors R times

In [7]:
gen_X(sample_size) = hcat(ones(sample_size), rand(Normal(5, 2), sample_size))


gen_X (generic function with 1 method)

In [8]:
gen_μ(sample_size) = randn(sample_size,1)*sqrt(σ²)

gen_μ (generic function with 1 method)

In [9]:
function MC_numeric!(β, sample_size)
    β̂̄_MC = zeros(length(β)) #it is beta-hat-bar
    se_β̂̄ = 0
    X = gen_X(sample_size) #deterministic
    for MC_run = 1:R
        μ = gen_μ(sample_size)
        σ² = dot(μ, μ) / (T - K) #DO THEY MEAN THIS BY TRUE STANDARD ERRORS?
        y_model = y(X,β,μ)
        gls_results = gls(y_model,X)
        β̂̄_MC = β̂̄_MC + gls_results.coefs/R
        se_β̂̄ = se_β̂̄ + sqrt(diag(gls_results.vcv))/R #I THINK THIS GIVES THE MEAN OF THE SE OF $\hat{\beta}$
    end
    return β̂̄_MC, se_β̂̄
end

MC_numeric! (generic function with 1 method)

In [10]:
MC_numeric!(β, sample_size)

LoadError: UndefVarError: T not defined

step 2: calculate statistics and save them (ols estimator, estimated ols standard error, t-statistic)

Note that std does not use the standard formulation of standard deviation

this gives the true standard error of the total simulation

In [11]:
std_of_x(x) = norm(x - mean(x))/sqrt(length(x))

std_of_x (generic function with 1 method)

In [12]:
t_test(vec,H₀) = (mean(vec) - H₀)/std(vec)

t_test (generic function with 1 method)

part 2: lagged dependent variable

Introducing lagged dependent variables makes it so that the assumption "X and $\mu$ are independent" has to be relaxed to $E[\mu_t|x_t] = 0$ or thus that the errors are contemporaneously independent with any explanatory variables.

The OLS estimator becomes:
* Biased: $E[\hat{\beta}|X] = \beta + (X'X)^{-1}X'E[\mu|X]$ => $E[\hat{\beta}] = E_X(E[\hat{\beta}|X]) \neq \beta$
* Consistent and asymptotically normally distributed: $plim\hat{\beta} = \beta + plim \frac{X'X}{T}^{-1} plim\frac{X'\mu}{T}$ = 0 because $plim\frac{X'\mu}{T} = E(x_t\mu_t) = 0$
* $\hat{\sigma}^2 = \frac{\hat{\mu}'\hat{\mu}}{T-k}$ is still a consistent estimator for $\sigma^2$

In [13]:
β₀ = 10
β₁ = 0
σ² = 1

1

In [14]:
y₀ = rand(Normal(β₀/(1-β₁), sqrt(σ²/(1-β₁^2)))) #keep fixed throughout sample sizes and drop from results
μ = randn(sample_size,1)*sqrt(σ²)

25×1 Array{Float64,2}:
 -0.792707
  1.58236 
  0.444177
 -0.749981
 -2.06984 
 -0.218239
  0.611709
  0.356565
  0.97148 
 -1.17465 
  0.13426 
  1.0049  
  1.43578 
 -1.72348 
  1.74055 
  0.664466
  0.330747
 -0.796498
 -0.653582
 -0.757919
 -0.574783
 -0.680621
 -0.773876
  1.35384 
  1.29551 

We need to generate a seed to kickstart the iterative time evolution

In [15]:
function gen_lagged_y!(y, β, y₀)
    β₀, β₁ = β
    
    μ = randn()
    y[1] = β₀ + β₁*y₀ + μ
    
    for t = 2:length(y)
        μ = randn()
        y[t] = β₀ + β₁*y[t-1] + μ
    end
    return y
end

gen_lagged_y! (generic function with 1 method)

In [16]:
function case2(sample_size, β, y₀)
    y = zeros(sample_size)
    return gen_lagged_y!(y, β,y₀)
end

case2 (generic function with 1 method)

In [17]:
case2(sample_size,β,y₀)

25-element Array{Float64,1}:
  21.0562
  31.4288
  39.5714
  49.9146
  59.8718
  70.3924
  81.294 
  89.8788
 100.065 
 110.661 
 121.103 
 132.343 
 143.775 
 153.151 
 163.34  
 173.938 
 184.196 
 194.989 
 206.925 
 217.56  
 228.851 
 237.695 
 246.768 
 256.071 
 265.603 

how can we use the MC simulation results to correct the bias of the OLS estimator for $\beta_1$?

* repeatedly save abs(true parameter - simulated parameter) to find the mean and distribution of the bias

In [18]:
OLS_normal(y, X) = inv(X'X) * X'y

OLS_normal (generic function with 1 method)

In [20]:
y_2 = case2(sample_size,β,y₀)
X = zeros(sample_size)
X[1] = y₀
for t = 1:(length(X)-1)
    X[t+1] = y_2[t]
end
β₁_sim = OLS_normal(y,hcat(ones(sample_size),X))[2]

LoadError: MethodError: no method matching *(::Array{Float64,2}, ::#y)[0m
Closest candidates are:
  *(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  *{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},S}(::Union{Base.ReshapedArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N}}},DenseArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2},SubArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:Union{Base.ReshapedArray{T,N,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{Int64}},N}},L}}, [1m[31m::Union{Base.ReshapedArray{S,1,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N}}},DenseArray{S,1},SubArray{S,1,A<:Union{Base.ReshapedAr